In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from css_dla import Model
import numpy as np
from scipy import stats

In [ ]:
model = Model(mode='single', w=1000, h=1000)

In [ ]:
import cProfile
import re
model = Model(mode='single', w=1000, h=1000)
# model.loop(1)
cProfile.run('model.loop(1000)', 'restats')

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats("focal_attraction")

In [ ]:
# plot the grid
plt.imshow(model.grid, cmap='Set1')
plt.clim(0,10)
plt.show()

In [ ]:
model.loop(4000)

In [ ]:
plt.imshow(model.grid)
plt.title("Final grid 2")
plt.show()

In [ ]:
distances, densities = model.density_gradient()

plt.plot(distances, densities, '-o', color='blue')
plt.xlabel('Distance from center')
plt.ylabel('Density')
plt.title('Density Gradient Plot')
plt.show()

In [ ]:
import powerlaw

# Fit the distance data using powerlaw
results = powerlaw.Fit(densities)

# Retrieve the parameters of the power-law distribution and comparison results
print('Power-law exponent alpha:', results.power_law.alpha)
print('Minimum value for power-law distribution xmin:', results.power_law.xmin)
R, p = results.distribution_compare('power_law', 'exponential')
print('Comparison result R between power-law and exponential distributions:', R)
print('p-value:', p)


In [ ]:
# Convert to log scale
log_distances = np.log(distances)
log_densities = np.log(densities)

plt.plot(log_distances, log_densities, '-o', color='blue')
plt.xlabel('Log(Distance from center)')
plt.ylabel('Log(Density)')
plt.title('Log-Log Density Gradient Plot')
fig, ax = plt.subplots()
results.plot_pdf(color='black', linewidth=2, label = 'PDF')
results.power_law.plot_pdf(color='b', ax=ax, label='Power-law Fit') 
results.exponential.plot_pdf(color='r', linestyle='--', ax=ax, label='Exponential Fit')  
results.lognormal.plot_pdf(color='g', linestyle='--', ax=ax, label='Lognormal Fit') 
plt.xlabel('Density')
plt.ylabel('Probability')
plt.title('Density Distribution and Power-law Fit')
plt.legend() 
plt.show()

In [ ]:
fractal_dimension = model.get_fractal_dim()

In [ ]:
print(fractal_dimension)


In [ ]:
res = []
for _ in range(10):
    model.loop(100)
    f = model.get_fractal_dim()
    res.append(f)
    print(f)


In [ ]:
density_res = []
fractal_res = []

num_runs = 10

for _ in range(num_runs):
    model.loop(100)
    _, densities = model.density_gradient()
    density_res.append(densities)
    
    fractal_dim = model.get_fractal_dim()
    fractal_res.append(fractal_dim)

    print(densities)
    print(fractal_dim)


In [ ]:
density_res = np.array(density_res)
fractal_res = np.array(fractal_res)

# calculate mean and 95% confidence interval for densities and fractal dimensions
mean_densities = np.mean(density_res, axis=0)
ci_densities = stats.sem(density_res, axis=0) * stats.t.ppf((1 + 0.95) / 2., num_runs - 1)

mean_fractal_dim = np.mean(fractal_res)
ci_fractal_dim = stats.sem(fractal_res) * stats.t.ppf((1 + 0.95) / 2., num_runs - 1)

In [ ]:
# plot for density gradient
plt.plot(mean_densities, label='Mean Density')
plt.fill_between(range(len(mean_densities)), 
                 mean_densities - ci_densities, 
                 mean_densities + ci_densities, 
                 color='red', alpha=0.2, label='95% Confidence Interval')
plt.title('Density Gradient with Confidence Interval')
plt.xlabel('Distance from Center')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
# plot for fractal dimensions
plt.boxplot(fractal_res, patch_artist=True)
plt.title('Boxplot of Fractal Dimensions')
plt.ylabel('Fractal Dimension')
plt.xticks([1], ['Fractal Dimensions'])
plt.grid(True)
plt.show()
